<a href="https://colab.research.google.com/github/jsansao/teic-20231/blob/main/TEIC_Licao27bis_B2W_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook Colab: Fine-Tuning do BERT (Português) para Análise de Sentimentos (B2W-Reviews)

Este notebook demonstra o processo de fine-tuning de um modelo BERT pré-treinado para a língua portuguesa (`neuralmind/bert-base-portuguese-cased`) para a tarefa de classificação de sentimentos.

Usaremos o dataset `b2w-reviews`, que contém reviews de produtos em português.

## Passo 1: Instalação e Configuração

Primeiro, instalamos as bibliotecas necessárias.

In [1]:
!pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00


In [2]:
# Importações principais
import torch
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
import evaluate

## Passo 2: Carregamento do Dataset (B2W-Reviews)

Carregamos o dataset `b2w-reviews`. Este dataset possui as colunas `review_text` (o texto) e `stars` (de 1 a 5).

In [4]:
# Carrega o dataset B2W-Reviews
dataset = load_dataset("vladjr/B2W-Reviews01")

print("Estrutura do dataset original:")
print(dataset)
print("\nExemplo de dado (antes do processamento):")
print(dataset["train"][0])

B2W-Reviews01.csv:   0%|          | 0.00/49.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/132373 [00:00<?, ? examples/s]

Estrutura do dataset original:
DatasetDict({
    train: Dataset({
        features: ['submission_date', 'reviewer_id', 'product_id', 'product_name', 'product_brand', 'site_category_lv1', 'site_category_lv2', 'review_title', 'overall_rating', 'recommend_to_a_friend', 'review_text', 'reviewer_birth_year', 'reviewer_gender', 'reviewer_state'],
        num_rows: 132373
    })
})

Exemplo de dado (antes do processamento):
{'submission_date': '2018-01-01 00:11:28', 'reviewer_id': 'd0fb1ca69422530334178f5c8624aa7a99da47907c44de0243719b15d50623ce', 'product_id': 132532965, 'product_name': 'Notebook Asus Vivobook Max X541NA-GO472T Intel Celeron Quad Core 4GB 500GB Tela LED 15,6" Windows - 10 Branco', 'product_brand': None, 'site_category_lv1': 'Informática', 'site_category_lv2': 'Notebook', 'review_title': 'Bom', 'overall_rating': 4, 'recommend_to_a_friend': 'Yes', 'review_text': 'Estou contente com a compra entrega rápida o único problema com as Americanas é se houver troca ou devolução do pro

## Passo 3: Pré-processamento e Mapeamento de Labels

O BERT se beneficia mais de tarefas de classificação claras. Vamos converter as 5 estrelas em 2 classes (Negativo/Positivo):

* **Negativo (label 0):** Estrelas 1 e 2
* **Positivo (label 1):** Estrelas 4 e 5
* **Neutro (Descartado):** Estrela 3 (para tornar a distinção entre classes mais clara)

Também renomearemos `review_text` para `text` para padronizar.

In [5]:
# 1. Filtra reviews neutros (3 estrelas)
dataset_filtered = dataset.filter(lambda example: example['overall_rating'] != 3)

# 2. Mapeia estrelas para labels (0 ou 1)
def map_labels(example):
    if example['overall_rating'] in [1, 2]:
        example['label'] = 0  # Negativo
    elif example['overall_rating'] in [4, 5]:
        example['label'] = 1  # Positivo
    return example

dataset_mapped = dataset_filtered.map(map_labels)

# 3. Renomeia a coluna de texto
dataset_renamed = dataset_mapped.rename_column("review_text", "text")

# 4. Remove colunas desnecessárias
columns_to_remove = ['overall_rating', 'submission_date', 'reviewer_id', 'product_id', 'product_name', 'product_brand', 'site_category_lv1', 'site_category_lv2', 'review_title', 'recommend_to_a_friend', 'reviewer_birth_year', 'reviewer_gender', 'reviewer_state']
#columns_to_remove = ['stars', 'submission_date', 'reviewer_id', 'product_id', 'review_title', 'product_name', 'product_brand', 'site_category_lv1', 'site_category_lv2', 'review_link']
dataset_final = dataset_renamed.remove_columns(columns_to_remove)

print("\nEstrutura do dataset processado:")
print(dataset_final)
print("\nExemplo de dado (após processamento):")
print(dataset_final["train"][5])

Filter:   0%|          | 0/132373 [00:00<?, ? examples/s]

Map:   0%|          | 0/116058 [00:00<?, ? examples/s]


Estrutura do dataset processado:
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 116058
    })
})

Exemplo de dado (após processamento):
{'text': 'Excelente produto, por fora em material acrílico super resistente e por dentro em adamantio, faz milagre com qualquer bebida. Sugiro aproveitarem a promoção antes que acabe.', 'label': 1}


## Passo 4: Amostragem (Treinamento Rápido)

O dataset B2W tem +130k reviews. Para um treinamento rápido de demonstração, vamos usar um subconjunto.

**Remova este passo (as 2 linhas abaixo) para treinar no dataset completo.**

In [6]:
# Take a smaller sample of the dataset for faster training (10%)
small_dataset = dataset_final["train"].shuffle(seed=42).select(range(int(len(dataset_final["train"]) * 0.01)))

# Split the small dataset into training and testing sets
# Using 80% of the small dataset for training and 20% for testing
train_test_split = small_dataset.train_test_split(test_size=0.2, seed=42)

train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]

print(f"Using {len(train_dataset)} examples for training.")
print(f"Using {len(test_dataset)} examples for testing.")

Using 928 examples for training.
Using 232 examples for testing.


In [7]:
import gc

# Delete the large dataset to free up memory
del dataset_final
del dataset_filtered
del dataset_mapped
del dataset_renamed
del dataset


# Run garbage collection
gc.collect()

print("Original dataset removed from memory.")

Original dataset removed from memory.


## Passo 5: Carregamento do Tokenizador e Modelo (Português)

Agora, usamos o `neuralmind/bert-base-portuguese-cased` como nosso modelo base.

* **`cased`**: Significa que o modelo diferencia maiúsculas de minúsculas.
* **`id2label`**: Mapeia os IDs das labels (0 e 1) para nomes legíveis ("Negativo", "Positivo"). Isso melhora a saída do `pipeline`.

In [10]:
# Checkpoint do modelo em português
model_checkpoint = "neuralmind/bert-base-portuguese-cased"

# Carrega o tokenizador
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)



# Carrega o modelo de classificação
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=2
)

tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Passo 6: Pré-processamento (Tokenização)

Esta função aplica o tokenizador aos nossos textos.

In [11]:
def tokenize_function(examples):
    # 'truncation=True' corta textos maiores que o limite máximo do modelo (512 para o BERT)
    # Explicitly add padding here when batched=True to ensure consistent shapes for the datasets library
    return tokenizer(examples["text"], truncation=True, max_length=512)

# Filter out examples with None values in 'text' before mapping
train_dataset_filtered = train_dataset.filter(lambda example: example["text"] is not None)
test_dataset_filtered = test_dataset.filter(lambda example: example["text"] is not None)

# Apply tokenization to the filtered datasets
tokenized_train_dataset = train_dataset_filtered.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset_filtered.map(tokenize_function, batched=True)


# The DataCollator will handle padding dynamically (more efficient)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Filter:   0%|          | 0/928 [00:00<?, ? examples/s]

Filter:   0%|          | 0/232 [00:00<?, ? examples/s]

Map:   0%|          | 0/905 [00:00<?, ? examples/s]

Map:   0%|          | 0/225 [00:00<?, ? examples/s]

## Passo 7: Definição da Métrica de Avaliação

Usaremos a Acurácia como métrica principal.

In [12]:
# Carrega a métrica de acurácia
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

## Passo 8: Configuração do Treinamento (Fine-Tuning)

Configuramos os `TrainingArguments` e o `Trainer`.

In [13]:
# Define os argumentos do treinamento
training_args = TrainingArguments(
    output_dir="./bert-b2w-finetuned",          # Onde salvar o modelo
    learning_rate=2e-5,                        # Taxa de aprendizado
    per_device_train_batch_size=8,             # Tamanho do batch de treino reduzido
    per_device_eval_batch_size=8,              # Tamanho do batch de avaliação reduzido
    num_train_epochs=3,                        # Número de épocas
    weight_decay=0.01,
    eval_strategy="epoch",               # Avaliar a cada época
    save_strategy="epoch",                     # Salvar a cada época
    load_best_model_at_end=True,               # Carregar o melhor modelo no final
    push_to_hub=False,
    # Adicionar gradient_accumulation_steps se reduzir o batch size não for suficiente
    # gradient_accumulation_steps=2, # Example: Accumulate gradients over 2 steps
)

# Cria o objeto Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

## Passo 9: Iniciar o Treinamento

**Certifique-se de estar em um ambiente com GPU!** (Vá em *Ambiente de execução > Alterar tipo de ambiente de execução > Acelerador de hardware > T4 GPU*).

In [14]:
# Inicia o fine-tuning!
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jsansao to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.274505,0.928889
2,No log,0.195190,0.955556
3,No log,0.206993,0.960000


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=342, training_loss=0.12322390149211326, metrics={'train_runtime': 3328.0255, 'train_samples_per_second': 0.816, 'train_steps_per_second': 0.103, 'total_flos': 122381071290660.0, 'train_loss': 0.12322390149211326, 'epoch': 3.0})

## Passo 10: Avaliação Final

Vejamos a acurácia final do nosso melhor modelo no conjunto de teste.

In [15]:
# Avalia o melhor modelo no dataset de teste
eval_results = trainer.evaluate()

print("\nResultados da Avaliação Final:")
print(eval_results)

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



Resultados da Avaliação Final:
{'eval_loss': 0.19518998265266418, 'eval_accuracy': 0.9555555555555556, 'eval_runtime': 87.5822, 'eval_samples_per_second': 2.569, 'eval_steps_per_second': 0.331, 'epoch': 3.0}


## Passo 11: Teste com Novas Frases (Inferência)

Vamos usar a ferramenta `pipeline` para testar nosso modelo recém-treinado com frases em português.

In [16]:
from transformers import pipeline

# Detecta se há GPU disponível
device = 0 if torch.cuda.is_available() else -1

# Carrega o pipeline de análise de sentimentos com nosso melhor modelo salvo
best_model_path = trainer.state.best_model_checkpoint
print(f"\nCarregando o melhor modelo salvo de: {best_model_path}")

sentiment_pipeline_pt = pipeline(
    "sentiment-analysis",
    model=best_model_path,
    tokenizer=tokenizer,
    device=device  # -1 para CPU, 0 para GPU
)

print("\n--- Teste de Inferência em Português ---")

review_pos = "Este produto é maravilhoso! Superou minhas expectativas, a entrega foi rápida e o material é de ótima qualidade."
review_neg = "Qualidade péssima, quebrou no primeiro dia. Não recomendo de jeito nenhum, estou muito decepcionado."
review_neu = "É um celular ok, funciona, mas a bateria não dura muito. Esperava mais pelo preço."

print(f"\nFrase: {review_pos}")
print(f"Resultado: {sentiment_pipeline_pt(review_pos)}")

print(f"\nFrase: {review_neg}")
print(f"Resultado: {sentiment_pipeline_pt(review_neg)}")

print(f"\nFrase: {review_neu}")
print(f"Resultado: {sentiment_pipeline_pt(review_neu)}")


Carregando o melhor modelo salvo de: ./bert-b2w-finetuned/checkpoint-228


Device set to use cpu



--- Teste de Inferência em Português ---

Frase: Este produto é maravilhoso! Superou minhas expectativas, a entrega foi rápida e o material é de ótima qualidade.
Resultado: [{'label': 'LABEL_1', 'score': 0.9983850717544556}]

Frase: Qualidade péssima, quebrou no primeiro dia. Não recomendo de jeito nenhum, estou muito decepcionado.
Resultado: [{'label': 'LABEL_0', 'score': 0.995383083820343}]

Frase: É um celular ok, funciona, mas a bateria não dura muito. Esperava mais pelo preço.
Resultado: [{'label': 'LABEL_0', 'score': 0.7106583118438721}]
